# Final Project

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this project are the following: 


In [1]:
#intial set of imports
import nltk
from nltk.corpus import wordnet, stopwords
from nltk.metrics import jaccard_distance, edit_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.wsd import lesk
import pandas as pd
import string
import regex

#variable initialization and instantiation
tests = []
tests_lem = []
gold_std_train = []
gold_std_test = []
lem1 = []
lem2 = []


## 1. Read all pairs of sentences of the train and test set
We proceed to open and read the input file

In [2]:
def text_to_sentences(filename):
    sentence_pair_array = []
    for line in open(filename, encoding="UTF8").readlines():
        sentence_pair_array.append([s.strip() for s in line.split("\t")])
    return sentence_pair_array

# TRIAL TESTING
trial_input = text_to_sentences('./00_data/trial/STS.input_fixed.txt')
trial_classes = open('./00_data/trial/STS_fixed.gs.txt', encoding="utf-8-sig").readlines()

# TRAINING PHASE
train_input = text_to_sentences('./00_data/train/STS.input.MSRpar_vid_SMT.txt')
train_classes = open('./00_data/train/STS.gs.MSRpar_vid_SMT.txt', encoding="utf-8-sig").readlines()

# TESTING PHASE
test_input = text_to_sentences('./00_data/test-gold/STS.input.ALL.txt')
test_classes = open('./00_data/test-gold/STS.gs.ALL.txt', encoding="UTF8").readlines()

We implement the lematizer. 

## 2. Noisy entities removal functions
### Stopwords, URL's, Punctuations

In [3]:
# Removing the punctuation and lowering the case of a string
def preprocessing(line):
    line = line.lower()
    # Clean the text
    line = regex.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", line)
    line = regex.sub(r"what's", "what is ", line)
    line = regex.sub(r"\'s", " ", line)
    line = regex.sub(r"\'ve", " have ", line)
    line = regex.sub(r"can't", "cannot ", line)
    line = regex.sub(r"n't", " not ", line)
    line = regex.sub(r"i'm", "i am ", line)
    line = regex.sub(r"\'re", " are ", line)
    line = regex.sub(r"\'d", " would ", line)
    line = regex.sub(r"\'ll", " will ", line)
    line = regex.sub(r",", " ", line)
    line = regex.sub(r"\.", " ", line)
    line = regex.sub(r"!", " ! ", line)
    line = regex.sub(r"\/", " ", line)
    line = regex.sub(r"\^", " ^ ", line)
    line = regex.sub(r"\+", " + ", line)
    line = regex.sub(r"\-", " - ", line)
    line = regex.sub(r"\=", " = ", line)
    line = regex.sub(r"'", " ", line)
    line = regex.sub(r"(\d+)(k)", r"\g<1>000", line)
    line = regex.sub(r":", " : ", line)
    line = regex.sub(r" e g ", " eg ", line)
    line = regex.sub(r" b g ", " bg ", line)
    line = regex.sub(r" u s ", " american ", line)
    line = regex.sub(r"\0s", "0", line)
    line = regex.sub(r" 9 11 ", "911", line)
    line = regex.sub(r"e - mail", "email", line)
    line = regex.sub(r"j k", "jk", line)
    line = regex.sub(r"\s{2,}", " ", line)

    #only accept alphanum
    # [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    #remove punctuation
    return line.translate(str.maketrans('', '', string.punctuation))

## 3. Word normalization
### Tokenization, Lemmatization

In [4]:
# extract the words from the sentence minus stopwords
def words_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word not in nltk.corpus.stopwords.words('english')]
    
# convert words to tokens
def tokens_from_words(words):
    return pos_tag(words)

# Function to get wordnet pos code
def wordnet_pos_code(tag):
    if tag.startswith('NN'):
        return wordnet.NOUN
    elif tag.startswith('VB'):
        return wordnet.VERB
    elif tag.startswith('JJ'):
        return wordnet.ADJ
    elif tag.startswith('RB'):
        return wordnet.ADV
    else:
        return None

# Tokens to lemmas using wordnet lemmatizer    
def tokens_to_lemmas(tokens):
    return list(map(token_to_lemmas, tokens))

def token_to_lemmas(token):    
    pos = wordnet_pos_code(token[1])
    if pos:
        return WordNetLemmatizer().lemmatize(token[0], pos=pos)
    return token[0]

## 4. Synset, Nammed Entity and Content Parsing


In [5]:
def lesking_sentence(pos_tagged_sentence):
    """
    Returns a sentence as the given sentece using lesker algorithms.
    The input sentence must be a pos_tagged sentence (e.g. [('The', 'DN'),
    ('sun', 'NN')]).
    """
    sentence = [i[0] for i in pos_tagged_sentence]
    result = []
    
    none_type_objects = []
    for word, tag in pos_tagged_sentence:
        # 'NoneType' object has no attribute 'name'
        try:
            result.append(lesk(sentence,word, wordnet_pos_code(tag)).name())
        except:
            result.append(word)            
    return result

## 5. Distance Metrics

## 6. Potential Networks
- MLPRegressor --> Using regressors
- Support Vector Regressor
- KNN Regressor

## 7. Evaluation

In [6]:
# evaluation imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
%matplotlib inline

## Main Program

In [7]:
def lemma(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    print(sent_0 + '\n' + sent_1 + '\n')
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    
    #jaccard_similarity
    print ('jaccard:', float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))), 'edit_dist:', float(edit_distance(lemmas_0, lemmas_1)))
    return [float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))),float(edit_distance(lemmas_0, lemmas_1))]

In [8]:
def lemma_levenshtein(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    print(sent_0 + '\n' + sent_1 + '\n')
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    
    #edit_distance
    print (float(edit_distance(lemmas_0, lemmas_1)), '\n')
    return float(edit_distance(lemmas_0, lemmas_1))


In [9]:
def lesk_jaccard(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    #print(sent_0 + '\n' + sent_1 + '\n')
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lesk_0, lesk_1 = lesking_sentence(tokens_0), lesking_sentence(tokens_1)
    
    #jaccard_similarity
    #print (float(1 - jaccard_distance(set(lesk_0), set(lesk_1))), '\n')
    return float(1 - jaccard_distance(set(lesk_0), set(lesk_1)))

In [10]:
#print('Training Lemmas')
#training_data_X_lemma_jaccard = [lemma(data[0], data[1])[0] for data in train_input]
#training_data_X_lemma_edit = [lemma(data[0], data[1])[1] for data in train_input]

In [11]:
print('Training Lemmas')
training_data_X_lemma_jaccard = [lemma(data[0], data[1])[0] for data in train_input]
training_data_X_lemma_edit = [lemma(data[0], data[1])[1] for data in train_input]

print('Training Lesk')
training_data_X_lesk = [lesk_jaccard(data[0], data[1]) for data in train_input]
training_scores_y = [float(line.strip()) for line in train_classes]
print('Finished Training!\n')

print('Testing Lemmas')
testing_data_X_lemma_jaccard = [lemma(data[0], data[1])[0] for data in test_input]
testing_data_X_lemma_edit = [lemma(data[0], data[1])[1] for data in test_input]

print('Testing Lesk')
testing_data_X_lesk = [lesk_jaccard(data[0], data[1])for data in test_input]

testing_scores_y = [float(line.strip()) for line in test_classes]
print('Finished Testing!\n')

print('Results')
#maybe label encoding https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous
#print(training_data_X)
#print('########################################################################\n')
#print(np.array(training_data_X).reshape(-1,1))
#print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n')
#print(training_scores_Y)


Training Lemmas
.but other sources close to the sale said vivendi was keeping the door open to further bids and hoped to see bidders interested in individual assets team up 
but other sources close to the sale said vivendi was keeping the door open for further bids in the next day or two 

jaccard: 0.4736842105263158 edit_dist: 7.0
.micron has declared its first quarterly profit for three years 
micron numbers also marked the first quarterly profit in three years for the dram manufacturer 

jaccard: 0.5 edit_dist: 5.0
.the fines are part of failed republican efforts to force or entice the democrats to return 
perry said he backs the senate efforts including the fines to force the democrats to return 

jaccard: 0.3571428571428571 edit_dist: 7.0
.the american anglican council which represents episcopalian conservatives said it will seek authorization to create a separate group 
the american anglican council which represents episcopalian conservatives said it will seek authorization to cr

jaccard: 0.3125 edit_dist: 11.0
.shares in eds closed on thursday at 18 51 a gain of 6 cents 
shares of eds closed thursday at 18 51 up 6 cents on the new york stock exchange 

jaccard: 0.6153846153846154 edit_dist: 5.0
.nationally the federal centers for disease control and prevention recorded 4 156 cases of west nile including 284 deaths 
there were 293 human cases of west nile in indiana in 2002 including 11 deaths statewide 

jaccard: 0.23809523809523814 edit_dist: 13.0
.a divided supreme court ruled monday that congress can force the nation public libraries to equip computers with anti  pornography filters 
the supreme court said monday the government can require public libraries to equip computers with anti  pornography filters rejecting librarians complaints that the law amounts to censorship 

jaccard: 0.36 edit_dist: 12.0
.the weakness exists in the way that vba looks at the properties of documents passed to it when the document is opened by a host application 
the vulnerabili

.site finder has been visited 65 million times since its introduction galvin said 
through sunday sept 21 site finder has been visited over 65 million times by internet users 

jaccard: 0.4 edit_dist: 7.0
.there are 103 democrats in the assembly and 47 republicans 
democrats dominate the assembly while republicans control the senate 

jaccard: 0.2222222222222222 edit_dist: 5.0
.mr pollard said  this is a terrible personal tragedy and a shocking blow for james family 
nick pollard the head of sky news said  this is a shocking blow for james family 

jaccard: 0.4285714285714286 edit_dist: 5.0
. i came basically to washington to establish relationships and to make sure that we are getting more federal money to california schwarzenegger said after meeting with congressional republicans 
 i came to washington basically to establish relationships and make sure we are getting more federal money schwarzenegger said after one meeting 

jaccard: 0.7647058823529411 edit_dist: 6.0
.the latest snap

jaccard: 0.625 edit_dist: 3.0
.get it all out says howard davidowitz chairman of davidowitz associates a national retail consulting firm based in new york city 
innocent or not she damaged goods said howard davidowitz chairman of davidowitz associates a national retail consulting firm in new york 

jaccard: 0.6470588235294117 edit_dist: 5.0
.the updated products include pylon pro pylon conduit pylon anywhere and pylon application server 
the new products on the desktop side include the latest versions of pylon conduit and pylon pro 

jaccard: 0.3571428571428571 edit_dist: 9.0
.dixon was otherwise the class of the field at pikes peak international raceway 
scott dixon eventually made winning the honda indy 225 look easy sunday at pikes peak international raceway 

jaccard: 0.21052631578947367 edit_dist: 11.0
.turkish authorities have said all the suicide bombers were turks 
ankara says all four suicide bombers were turkish 

jaccard: 0.5 edit_dist: 4.0
.in april it had forecast operatin

jaccard: 0.6666666666666667 edit_dist: 7.0
.joining stern and mcentee on stage was international union of painters and allied trades president james williams 
the international union of painters and allied trades endorsed mr dean several weeks ago 

jaccard: 0.2777777777777778 edit_dist: 10.0
.it exploded in his hands but the former italian prime minister was unhurt 
the letter bomb sent to prodi exploded in his hands but he was unhurt 

jaccard: 0.2727272727272727 edit_dist: 6.0
.some 95 million americans   half of all households   invest in mutual funds 
about half of all american households have money in mutual funds 

jaccard: 0.5555555555555556 edit_dist: 4.0
.halabi military attorney air force maj james key denied the charges which could carry a death penalty 
the attorney representing al  halabi air force maj james key iii denied the charges according to the associated press 

jaccard: 0.44999999999999996 edit_dist: 9.0
.waksal in a letter to the court said  i tore my family apa

...

jaccard: 0.8333333333333334 edit_dist: 2.0
.after all they recommend the policies that politicians may or may not want to consider 
after all they recommended the policies which the politicians take or not into account 

jaccard: 0.2222222222222222 edit_dist: 5.0
.today there are two plausible ways to proceed against a deposed tyrant 
today there are two ways of proceeding to the origins 

jaccard: 0.5 edit_dist: 3.0
.in other areas sharia law is being introduced by force 
in other areas the sharia is imposed 

jaccard: 0.33333333333333337 edit_dist: 3.0
.but if europe performance continues to decline would this perception survive 
but this picture can survive in a european 

jaccard: 0.11111111111111116 edit_dist: 7.0
.this tendency extends deeper than headscarves 
this trend goes well beyond simple headscarves 

jaccard: 0.11111111111111116 edit_dist: 5.0
.but like the union itself it will be built and it will be done 
but as the european union itself build and happen 

jaccard: 0.33

jaccard: 0.6666666666666667 edit_dist: 1.0
.pro  market economists do not object to corporations that blatantly use snob appeal to promote their products 
economists pro  march are not opposed to companies which openly using the attractiveness of luxury to promote their products 

jaccard: 0.2941176470588235 edit_dist: 8.0
.gays and other modern practices are rejected as self  indulgent 
the gay and other practices modern are rejected because navel  gazing 

jaccard: 0.5 edit_dist: 4.0
.but they were necessary 
but they were necessary 

jaccard: 1.0 edit_dist: 0.0
.the political stock market
the politicization of securities markets

jaccard: 0.19999999999999996 edit_dist: 2.0
.the old version of the european response   what psychologists might call dollar envy   will only become more acute 
the old version of the european reaction what psychologists call the envy of the dollar will only more pervasive 

jaccard: 0.5384615384615384 edit_dist: 5.0
.the president has fewer powers than it 

jaccard: 0.125 edit_dist: 9.0
.a prominent chinese businessman who divides his time between hong  kong and london was even more specific 
an influential chinese businessman who shares his time between hong kong and london was even more specific 

jaccard: 0.6666666666666667 edit_dist: 2.0
.in other areas sharia law is being introduced by force 
in other areas sharia law is imposed 

jaccard: 0.5 edit_dist: 2.0
.he did but the initiative did not get very far 
what it has done without initiative go very far 

jaccard: 0.33333333333333337 edit_dist: 3.0
.this has left the conservative opposition without an effective response 
that leaves the opposition and without any effective response 

jaccard: 0.5714285714285714 edit_dist: 2.0
.but if europe performance continues to decline would this perception survive 
but this image further decline can survive 

jaccard: 0.25 edit_dist: 5.0
.but america interest in iraqi oil was not driven either by economics or energy policy 
but the interests of 

jaccard: 0.3571428571428571 edit_dist: 5.0
.only a month ago mubarak dismissed demands for constitutional reform as futile 
only a month ago muybarak cartait for constitutional reform in the taxing of futile 

jaccard: 0.4545454545454546 edit_dist: 4.0
.the book success is itself a sign of a kind of malaise 
the success of the book constitutes in itself a sign of faintness 

jaccard: 0.4285714285714286 edit_dist: 4.0
.other ways are needed 
we must find other ways 

jaccard: 0.25 edit_dist: 3.0
.jobs in particular will help young people to create new lives through serious work 
employment in particular would help the young people to forge new lives thanks to serious employment 

jaccard: 0.5 edit_dist: 5.0
.putin too also seems to suspect his regime impotence 
putin too seems suspicion powerlessness of his regime 

jaccard: 0.375 edit_dist: 4.0
.this gross error is leading russia to political ruin 
and this great mistake is russia to ruin its policy 

jaccard: 0.2222222222222222 edit_d

.this tendency extends deeper than headscarves 
this trend goes well beyond simple scarves 

jaccard: 0.0 edit_dist: 6.0
.indeed intolerance goes right to the top of the turkish government 
it is undeniable that intolerance reached until the heights of the turkish government 

jaccard: 0.30000000000000004 edit_dist: 4.0
.indeed the french philosopher marcel gauchet entitled a recent book democracy against itself 
we can also note that the french philosopher marcel gauchet has called one of these recent books democracy against itself 

jaccard: 0.5384615384615384 edit_dist: 4.0
.being a muslim and being an islamist are not the same thing 
to be muslim and islamic are two different things 

jaccard: 0.33333333333333337 edit_dist: 3.0
.in other areas sharia law is being introduced by force 
in other regions the sharia is imposed 

jaccard: 0.1428571428571429 edit_dist: 4.0
.indeed the french philosopher marcel gauchet entitled a recent book democracy against itself 
one can also note that

In [15]:
# Creating a Pandas DataFrame
df_X = pd.DataFrame(training_data_X_lemma_jaccard)
df_X["trainning lemma edit"] = pd.DataFrame(training_data_X_lemma_edit)
#df_X = pd.DataFrame(training_data_X_lesk)
#df_X["trainning lesk"] = pd.DataFrame(training_data_X_lesk)

df_y = pd.Series(training_scores_y)

df_X_Test = pd.DataFrame(testing_data_X_lemma_jaccard)
#df_X_Test = pd.DataFrame(testing_data_X_lesk)
#df_X_Test["testing lesk"] = pd.DataFrame(testing_data_X_lesk)
df_X_Test["testing lemma edit"] = pd.DataFrame(testing_data_X_lemma_edit)
df_X.head()

,0,trainning lemma edit
0,0.473684,7.0
1,0.500000,5.0
2,0.357143,7.0
3,0.611111,6.0
4,0.111111,14.0


In [28]:
from sklearn.neural_network import MLPRegressor
r = MLPRegressor(max_iter=1000)
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
a = pearsonr(training_scores_y, train_prediction)[0]
b = pearsonr(testing_scores_y, test_prediction)[0]
print('MLP Training Accuracy: ',round(a,3))
print('MLP Testing Accuracy: ', round(b,3))
print('MLP Drop Train-Test: ', round(a-b,3))

Results are ready!

MLP Training Accuracy:  0.762
MLP Testing Accuracy:  0.655
MLP Drop Train-Test:  0.107


In [26]:
from sklearn.svm import SVR
r = SVR(gamma='scale', C=1.0, epsilon=0.2)
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
a = pearsonr(training_scores_y, train_prediction)[0]
b = pearsonr(testing_scores_y, test_prediction)[0]
print('SVR Training Accuracy: ',round(a,3))
print('SVR Testing Accuracy: ', round(b,3))
print('SVR Drop Train-Test: ', round(a-b,3))

Results are ready!

SVR Training Accuracy:  0.727
SVR Testing Accuracy:  0.579
SVR Drop Train-Test:  0.148


In [27]:
from sklearn.neighbors import KNeighborsRegressor
r = KNeighborsRegressor(n_neighbors=15)
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
a = pearsonr(training_scores_y, train_prediction)[0]
b = pearsonr(testing_scores_y, test_prediction)[0]
print('KNN Training Accuracy: ',round(a,3))
print('KNN Testing Accuracy: ', round(b,3))
print('KNN Drop Train-Test: ', round(a-b,3))

Results are ready!

KNN Training Accuracy:  0.787
KNN Testing Accuracy:  0.641
KNN Drop Train-Test:  0.146
